In [ ]:
# 패키지 로드
import geopandas as gpd
import pandas as pd
import os

In [ ]:
# GDAL_DATA 환경 변수 경로 설정
os.environ['GDAL_DATA'] = r'C:\Users\TG\miniconda3\envs\moonsoo1\Library\share\gdal'

In [26]:
# 파일 경로 설정
emd_path = 'D:/GIS/행정구역/읍면동/emd.shp'  # 읍면동 shp 파일 경로
li_path = 'D:/GIS/행정구역/리/li.shp'        # 리 shp 파일 경로

# shp 파일 읽기(한글 속성값 때문에 EUC-KR 인코딩 지정)
gdf_emd = gpd.read_file(emd_path, encoding='euc-kr')
gdf_li = gpd.read_file(li_path, encoding='euc-kr')

In [27]:
# 읍면동 shp 파일에서 '읍면동명' 컬럼(EMD_KOR_NM)의 값이 '동' 또는 '가'로 끝나는 행만 필터링
filtered_emd = gdf_emd[gdf_emd['EMD_KOR_NM'].str.endswith(('동', '가'))]

filtered_emd

,EMD_CD,EMD_ENG_NM,EMD_KOR_NM,geometry
0,11110101,Cheongun-dong,청운동,"POLYGON ((953700.022 1954605.065, 953693.871 1..."
1,11110102,Singyo-dong,신교동,"POLYGON ((953233.465 1953996.984, 953235.183 1..."
2,11110103,Gungjeong-dong,궁정동,"POLYGON ((953560.228 1954257.466, 953561.19 19..."
3,11110104,Hyoja-dong,효자동,"POLYGON ((953519.843 1953890.785, 953518.489 1..."
4,11110105,Changseong-dong,창성동,"POLYGON ((953516.123 1953734.362, 953516.526 1..."
...,...,...,...,...
4959,52210126,Myeongdeok-dong,명덕동,"POLYGON ((941613.041 1756499.437, 941612.117 1..."
4960,52210127,Jewol-dong,제월동,"POLYGON ((944892.699 1754219.699, 944893.429 1..."
4961,52210128,Dojang-dong,도장동,"POLYGON ((949481.601 1753490.356, 949488.931 1..."
4962,52210129,Seojeong-dong,서정동,"POLYGON ((948699.374 1752674.742, 948742.221 1..."


In [30]:
# 두 GeoDataFrame의 좌표계가 동일한지 확인하고, 다르면 변환 필요
if filtered_emd.crs != gdf_li.crs:
    gdf_li = gdf_li.to_crs(filtered_emd.crs)

In [ ]:
# EMD 파일의 코드는 EMD_CD 값 뒤에 '00'을 추가하고, 한국어 이름 컬럼을 'DONGLI'로 통일
filtered_emd['CODE'] = filtered_emd['EMD_CD'].astype(str) + "00"
filtered_emd = filtered_emd.rename(columns={'EMD_KOR_NM': 'DONGLI'})

# LI 파일은 LI_CD와 LI_KOR_NM을 각각 'CODE'와 'DONGLI'으로 변경
gdf_li = gdf_li.rename(columns={'LI_CD': 'CODE', 'LI_KOR_NM': 'DONGLI'})

# 불필요한 영어 이름 컬럼 제거 후, 필요한 컬럼만 선택(코드, 동리명, 지오메트리)
filtered_emd = filtered_emd[['CODE', 'DONGLI', 'geometry']]
gdf_li = gdf_li[['CODE', 'DONGLI', 'geometry']]

c:\Users\TG\miniconda3\envs\moonsoo1\Lib\site-packages\geopandas\geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [34]:
# 두 데이터를 병합(속성 및 지오메트리 모두 병합됨)
combined_gdf = gpd.GeoDataFrame(pd.concat([filtered_emd, gdf_li], ignore_index=True),
                                crs=filtered_emd.crs)

In [57]:
# 법정동코드 전체자료 파일 읽기 (탭 구분, 한글 인코딩 EUC-KR)
code_df = pd.read_csv('E:/Coding/TMDL/주소검토/법정동코드 전체자료.txt', sep='\t', encoding='euc-kr')

# '법정동코드' 컬럼을 문자열로 변환
code_df['법정동코드'] = code_df['법정동코드'].astype(str)

# '폐지여부' 컬럼의 값이 '존재'인 행만 필터링
code_df = code_df[code_df['폐지여부'] == '존재']

# '폐지여부' 컬럼 삭제
code_df = code_df.drop(columns='폐지여부')

code_df

,법정동코드,법정동명
0,1100000000,서울특별시
1,1111000000,서울특별시 종로구
2,1111010100,서울특별시 종로구 청운동
3,1111010200,서울특별시 종로구 신교동
4,1111010300,서울특별시 종로구 궁정동
...,...,...
49854,5280042024,전북특별자치도 부안군 위도면 대리
49855,5280042025,전북특별자치도 부안군 위도면 거륜리
49856,5280042026,전북특별자치도 부안군 위도면 식도리
49857,5280042027,전북특별자치도 부안군 위도면 상왕등리


In [60]:
# 법정동명 분할 함수 정의(3번째 항목이 '구'로 끝나면 2번째 항목과 결합)
def split_code_name(name):
    tokens = name.split()
    # tokens가 최소 3개 이상 있고, 세번째 토큰이 '구'로 끝나는 경우
    if len(tokens) >= 3 and tokens[2].endswith("구"):
        # 두번째 토큰과 세번째 토큰을 결합한 후, 세번째 토큰 제거
        tokens[1] = tokens[1] + "_" + tokens[2]
        tokens.pop(2)
    # 시도, 시군구, 읍면동에 해당하는 최소 3개 토큰을 확보 (부족하면 None 할당)
    while len(tokens) < 3:
        tokens.append(None)
    # 네번째 토큰이 존재하면 '리' 컬럼에 할당, 없으면 None
    result = {
        "시도": tokens[0],
        "시군구": tokens[1],
        "읍면동": tokens[2],
        "리": tokens[3] if len(tokens) > 3 else None
    }
    return pd.Series(result)

In [61]:
# '법정동명' 컬럼을 위 함수로 처리하여 새로운 컬럼 생성
code_df[['시도', '시군구', '읍면동', '리']] = code_df['법정동명'].apply(split_code_name)

# 결과 확인 (상위 5개 행)
code_df.head()

,법정동코드,법정동명,시도,시군구,읍면동,리
0,1100000000,서울특별시,서울특별시,None,None,None
1,1111000000,서울특별시 종로구,서울특별시,종로구,None,None
2,1111010100,서울특별시 종로구 청운동,서울특별시,종로구,청운동,None
3,1111010200,서울특별시 종로구 신교동,서울특별시,종로구,신교동,None
4,1111010300,서울특별시 종로구 궁정동,서울특별시,종로구,궁정동,None


In [ ]:
# combined_gdf와 법정동코드 전체자료를 'CODE'와 '법정동코드'를 키로 left join 수행
merged_gdf = combined_gdf.merge(code_df, left_on='CODE', right_on='법정동코드', how='left')

# 'CODE' 컬럼 삭제
merged_gdf = merged_gdf.drop(columns='CODE')

# 병합 결과 확인
merged_gdf.head()

,CODE,DONGLI,geometry,법정동코드,법정동명,시도,시군구,읍면동,리
0,1111010100,청운동,"POLYGON ((953700.022 1954605.065, 953693.871 1...",1111010100,서울특별시 종로구 청운동,서울특별시,종로구,청운동,None
1,1111010200,신교동,"POLYGON ((953233.465 1953996.984, 953235.183 1...",1111010200,서울특별시 종로구 신교동,서울특별시,종로구,신교동,None
2,1111010300,궁정동,"POLYGON ((953560.228 1954257.466, 953561.19 19...",1111010300,서울특별시 종로구 궁정동,서울특별시,종로구,궁정동,None
3,1111010400,효자동,"POLYGON ((953519.843 1953890.785, 953518.489 1...",1111010400,서울특별시 종로구 효자동,서울특별시,종로구,효자동,None
4,1111010500,창성동,"POLYGON ((953516.123 1953734.362, 953516.526 1...",1111010500,서울특별시 종로구 창성동,서울특별시,종로구,창성동,None


In [63]:
# 결과를 shp 파일로 저장(EUC-KR 인코딩 지정)
merged_gdf.to_file('D:/GIS/행정구역/리/dong_li.shp', encoding='euc-kr')

In [65]:
# 강원도만 추출
gangwon = merged_gdf[merged_gdf['시도'] == '강원특별자치도']

gangwon

,CODE,DONGLI,geometry,법정동코드,법정동명,시도,시군구,읍면동,리
3222,5111010100,봉의동,"POLYGON ((1019867.069 1987393.48, 1019867.069 ...",5111010100,강원특별자치도 춘천시 봉의동,강원특별자치도,춘천시,봉의동,None
3223,5111010200,요선동,"POLYGON ((1019955.386 1987141.041, 1019955.409...",5111010200,강원특별자치도 춘천시 요선동,강원특별자치도,춘천시,요선동,None
3224,5111010300,낙원동,"POLYGON ((1019635 1986381.326, 1019566.366 198...",5111010300,강원특별자치도 춘천시 낙원동,강원특별자치도,춘천시,낙원동,None
3225,5111010400,중앙로1가,"POLYGON ((1020217.642 1987072.283, 1020190.311...",5111010400,강원특별자치도 춘천시 중앙로1가,강원특별자치도,춘천시,중앙로1가,None
3226,5111010500,중앙로2가,"POLYGON ((1019733.606 1986437.275, 1019725.206...",5111010500,강원특별자치도 춘천시 중앙로2가,강원특별자치도,춘천시,중앙로2가,None
...,...,...,...,...,...,...,...,...,...
17344,5183035038,장산리,"POLYGON ((1094783.88 2014877.489, 1094802.091 ...",5183035038,강원특별자치도 양양군 강현면 장산리,강원특별자치도,양양군,강현면,장산리
17345,5183035039,전진리,"POLYGON ((1098120.453 2014314.88, 1098129.268 ...",5183035039,강원특별자치도 양양군 강현면 전진리,강원특별자치도,양양군,강현면,전진리
17346,5183035040,주청리,"POLYGON ((1098517.017 2013399.913, 1098517.928...",5183035040,강원특별자치도 양양군 강현면 주청리,강원특별자치도,양양군,강현면,주청리
17347,5183035041,적은리,"POLYGON ((1095184.931 2014773.485, 1095211.126...",5183035041,강원특별자치도 양양군 강현면 적은리,강원특별자치도,양양군,강현면,적은리


In [66]:
# 결과를 shp 파일로 저장(EUC-KR 인코딩 지정)
gangwon.to_file('D:/GIS/행정구역/리/dong_li_gangwon.shp', encoding='euc-kr')